In [1]:
from os import path

import tensorflow as tf

from src.utils import decode_image_record

processed = path.join('data', 'processed')

features = {
    'image': tf.io.FixedLenFeature([], tf.string),
    'label': tf.io.FixedLenFeature([2], tf.int64)
}
shape = (32, 32, 1)

def decoder(example):
    feature = tf.io.parse_single_example(example, features)
    image = tf.io.parse_tensor(feature['image'], tf.float32)
    image.set_shape(shape)
    # We only want the 'label_idx'. Not the 'angle'.
    label = feature['label'][0]

    return [image, label]


train_dataset = decode_image_record(path.join(processed, 'train.tfrecord'), decoder)
validation_dataset = decode_image_record(path.join(processed, 'validate.tfrecord'), decoder)
test_dataset = decode_image_record(path.join(processed, 'test.tfrecord'), decoder)

model_path = path.join('models', 'symbol_classifier','model.h5')
model = tf.keras.models.load_model(model_path)

print(model.evaluate(train_dataset, steps=100, verbose=0))
print(model.evaluate(validation_dataset, steps=100, verbose=0))
print(model.evaluate(test_dataset, steps=100, verbose=0))

[0.026712066430482084, 0.9934375]
[0.0784420772548765, 0.98625]
[0.15946074941544794, 0.9528125]
